## Configuration Details

In [1]:
from platform import python_version
print("VERSION: ", python_version()) # expect 3.7.0

In [113]:
# data science standards
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
from matplotlib.pyplot import figure
import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# keras/tensorflow
from tensorflow.metrics import auc as tf_auc
from tensorflow import local_variables_initializer
import keras
import keras.backend as K
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Conv2D, \
        MaxPooling1D, LSTM, Flatten, BatchNormalization,Embedding,Reshape, Dropout


# Local custom data loading functions
import load_data
import clean_data
import custom_feature_extraction
import custom_keras_metrics

In [3]:
# dirty spaCy error workaround:
get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""

### Load annoations

In [4]:
annotations ='../data/Annotations04-09-19.json'
df = load_data.getJSONData(annotations)

to  = 'label'
field = 'annotation'
df[to] = df.apply(lambda row:clean_data.convertAnnotationtoBinary(row, field), axis =1)   
df.head()

,annotation,fileID,text,label
0,NON_permission_statement.,1,a copy of this entire consent form will be giv...,0
1,permission_statement,1,i give my permission for photographs/audio/vid...,1
2,NON_permission_statement.,1,language for required recordings: the research...,0
3,NON_permission_statement.,1,"""(if applicable, add) information about indivi...",0
4,NON_permission_statement.,1,this consent form will be filed securely in an...,0


### Data Pre-processing

In [5]:
slim2 = df[['label', 'text']] # subset dataframe to make it easier to work with

# TODO: Functionalize

In [6]:
vocabulary_size = 1000
maxlen = 100

tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(slim2['text'])
sequences = tokenizer.texts_to_sequences(slim2['text'])
data = pad_sequences(sequences, maxlen=maxlen)

In [7]:
slim2['data'] = data.tolist()
# slim2['data'].head(10)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
new_col_names = ['seq_posi' + str(i) for i in range(0,maxlen)]
pos_seq_df = pd.DataFrame(slim2['data'].values.tolist(), columns=new_col_names)
slim2 = pd.concat([slim2, pos_seq_df], axis=1)
# slim2.head()

### Train-Test Split


In [9]:
text_train, text_test = train_test_split(slim2,
                               stratify=slim2['label'],
                               test_size=0.3, 
                               random_state=1729)

### Convert Labels to Categorical

In [10]:
label_train = keras.utils.to_categorical(text_train['label'], num_classes=2)
label_test = keras.utils.to_categorical(text_test['label'], num_classes=2)

## Baseline Models

In [11]:
models = []

dtBase = DecisionTreeClassifier(max_depth=10, 
                               max_features=9,
                               class_weight={1:.2})

models.append(("DecisionTree",dtBase))

rdfBase = RandomForestClassifier(n_estimators=1000,
                                class_weight={1:.2})

models.append(("RandomForest",rdfBase))

models.append(("SVM",SVC(gamma='auto')))
models.append(("LogisticRegression", LogisticRegression(solver='liblinear',
                                  max_iter=1000,
                                  penalty='l1')))


In [12]:
fitted_models = []
new_rows = []

for name, model in models:
    model.fit(text_train[new_col_names], text_train['label'])
    fitted_models.append((name, model))
    prediction_vec = model.predict(text_test[new_col_names])
    row = {
        'Model:': name,
        'Accuracy:': accuracy_score(text_test['label'], prediction_vec),
        'Precision:': precision_score(text_test['label'], prediction_vec),
        'ROC:': roc_auc_score(text_test['label'], prediction_vec)
    }
    
    new_rows.append(row)

baseline_results_df = pd.DataFrame(new_rows)

In [13]:
# Naive baseline
majority_class = np.zeros(len(text_test['label']))

count = len(baseline_results_df)

ac = accuracy_score(text_test['label'], majority_class)
pr = precision_score(text_test['label'], majority_class)
roc = roc_auc_score(text_test['label'], majority_class)

baseline_results_df.loc[count, 'Model:'] = 'Naive Majority Class Classifier'
baseline_results_df.loc[count, 'Accuracy:'] = ac
baseline_results_df.loc[count, 'Precision:'] = pr
baseline_results_df.loc[count, 'ROC:'] = roc

cols = ['Model:', 'Accuracy:', 'Precision:', 'ROC:']

baseline_results_df = baseline_results_df[cols]
baseline_results_df.head(len(baseline_results_df))

/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


,Model:,Accuracy:,Precision:,ROC:
0,DecisionTree,0.768904,0.272727,0.502707
1,RandomForest,0.786746,0.603774,0.549593
2,SVM,0.796092,0.866667,0.547433
3,LogisticRegression,0.772302,0.333333,0.504893
4,Naive Majority Class Classifier,0.777400,0.000000,0.500000


## Shared Hyperparameters

In [69]:
n_epochs = 5
to_shuffle = True
n_batch = 3

## Original CNN (Model 1)

In [14]:
cnn_1 = Sequential()
cnn_1.add(Embedding(vocabulary_size, 100, input_length=maxlen))
cnn_1.add(Dropout(0.2))
cnn_1.add(Conv1D(64, 5, activation='relu'))
cnn_1.add(MaxPooling1D(pool_size=4))
cnn_1.add(LSTM(100))
cnn_1.add(Dense(2, activation='softmax'))
cnn_1.compile(loss='binary_crossentropy', 
               optimizer='adam', 
                metrics=['accuracy', 
                    custom_keras_metrics.keras_precision, 
                    custom_keras_metrics.keras_recall, 
                    custom_keras_metrics.keras_auc])

fit_cnn_1 = cnn_1.fit(text_train[new_col_names], 
                  label_train, 
                  validation_data=(text_test[new_col_names],label_test), 
                  epochs=n_epochs,
                  shuffle=to_shuffle,
                  batch_size=3)
    

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use tf.cast instead.
Train on 2746 samples, validate on 1177 samples
Epoch 1/5
2746/2746 [==============================] - 23s 8ms/step - loss: 0.4049 - acc: 0.8281 - keras_precision: 0.8281 - keras_recall: 0.8281 - keras_auc: 0.8646 - val_loss: 0.3230 - val_acc: 0.8598 - val_keras_precision: 0.8598 - val_keras_recall: 0.8598 - val_keras_auc: 0.9052
Epoch 2/5
2746/2746 [==============================] - 25s 9ms/step - loss: 0.2607 - acc: 0.8973 - keras_precision: 0.8973 - keras_recall: 0.8973 - keras_auc: 0.9235 - val_loss: 0.3795 - val_acc: 0.8437 - val_keras_precision: 0.8437 - val_keras_recall: 0.8437 - val_keras_auc: 0.9315
Epoch 3/5
2746/2746 [==

## CNN 2 - Without LSTM

In [15]:
cnn_2 = Sequential()
cnn_2.add(Embedding(vocabulary_size, 100, input_length=maxlen))
cnn_2.add(Dropout(0.2))
cnn_2.add(Conv1D(64, 5, activation='relu'))
cnn_2.add(MaxPooling1D(pool_size=4))
cnn_2.add(Flatten())
cnn_2.add(Dense(2, activation='softmax'))
cnn_2.compile(loss='binary_crossentropy', 
               optimizer='adam', 
                metrics=['accuracy', 
                    custom_keras_metrics.keras_precision, 
                    custom_keras_metrics.keras_recall, 
                    custom_keras_metrics.keras_auc])

fit_cnn_2 = cnn_2.fit(text_train[new_col_names], 
                  label_train, 
                  validation_data=(text_test[new_col_names],label_test), 
                  epochs=n_epochs,
                  shuffle=to_shuffle,
                  batch_size=n_batch)

Train on 2746 samples, validate on 1177 samples
Epoch 1/5
2746/2746 [==============================] - 5s 2ms/step - loss: 0.4226 - acc: 0.8186 - keras_precision: 0.8186 - keras_recall: 0.8186 - keras_auc: 0.8405 - val_loss: 0.3259 - val_acc: 0.8709 - val_keras_precision: 0.8709 - val_keras_recall: 0.8709 - val_keras_auc: 0.8974
Epoch 2/5
2746/2746 [==============================] - 4s 1ms/step - loss: 0.2454 - acc: 0.9024 - keras_precision: 0.9024 - keras_recall: 0.9024 - keras_auc: 0.9219 - val_loss: 0.3491 - val_acc: 0.8666 - val_keras_precision: 0.8666 - val_keras_recall: 0.8666 - val_keras_auc: 0.9322
Epoch 3/5
2746/2746 [==============================] - 4s 1ms/step - loss: 0.1434 - acc: 0.9465 - keras_precision: 0.9465 - keras_recall: 0.9465 - keras_auc: 0.9418 - val_loss: 0.4155 - val_acc: 0.8632 - val_keras_precision: 0.8632 - val_keras_recall: 0.8632 - val_keras_auc: 0.9485
Epoch 4/5
2746/2746 [==============================] - 3s 1ms/step - loss: 0.0709 - acc: 0.9745 - keras

## CNN 3 - LSTM without Convolution

In [16]:
cnn_3 = Sequential()
cnn_3.add(Embedding(vocabulary_size, 100, input_length=maxlen))
cnn_3.add(Dropout(0.2))
cnn_3.add(LSTM(100))
cnn_3.add(Dense(2, activation='softmax'))
cnn_3.compile(loss='binary_crossentropy', 
               optimizer='adam', 
                metrics=['accuracy', 
                    custom_keras_metrics.keras_precision, 
                    custom_keras_metrics.keras_recall, 
                    custom_keras_metrics.keras_auc])

fit_cnn_3 = cnn_3.fit(text_train[new_col_names], 
                  label_train, 
                  validation_data=(text_test[new_col_names],label_test), 
                  epochs=n_epochs,
                  shuffle=to_shuffle,
                  batch_size=n_batch)

Train on 2746 samples, validate on 1177 samples
Epoch 1/5
2746/2746 [==============================] - 85s 31ms/step - loss: 0.4323 - acc: 0.8179 - keras_precision: 0.8179 - keras_recall: 0.8179 - keras_auc: 0.8437 - val_loss: 0.3368 - val_acc: 0.8649 - val_keras_precision: 0.8649 - val_keras_recall: 0.8649 - val_keras_auc: 0.8918
Epoch 2/5
2746/2746 [==============================] - 72s 26ms/step - loss: 0.2897 - acc: 0.8827 - keras_precision: 0.8827 - keras_recall: 0.8827 - keras_auc: 0.9124 - val_loss: 0.3339 - val_acc: 0.8590 - val_keras_precision: 0.8590 - val_keras_recall: 0.8590 - val_keras_auc: 0.9225
Epoch 3/5
2746/2746 [==============================] - 77s 28ms/step - loss: 0.2312 - acc: 0.9093 - keras_precision: 0.9093 - keras_recall: 0.9093 - keras_auc: 0.9313 - val_loss: 0.3461 - val_acc: 0.8522 - val_keras_precision: 0.8522 - val_keras_recall: 0.8522 - val_keras_auc: 0.9363
Epoch 4/5
2746/2746 [==============================] - 78s 28ms/step - loss: 0.1816 - acc: 0.9381

## Autoencoder

In [17]:
dim = text_train[new_col_names].shape[1]

auto_encoder_1 = Sequential()
auto_encoder_1.add(Dense(units=100, activation='relu', input_dim=dim))
auto_encoder_1.add(Dense(units=500, activation='softmax'))
auto_encoder_1.add(Dense(units=10, activation='softmax'))
auto_encoder_1.add(Dense(units=1000, activation='relu'))
auto_encoder_1.add(Dense(units=500, activation='relu'))
auto_encoder_1.add(Dense(units=2, activation='softmax'))

auto_encoder_1.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adagrad(lr=0.001), 
              metrics=['accuracy', 
                       custom_keras_metrics.keras_precision, 
                       custom_keras_metrics.keras_recall, 
                       custom_keras_metrics.keras_auc])

fit_auto_encoder_1 = auto_encoder_1.fit(text_train[new_col_names], 
                  label_train, 
                  validation_data=(text_test[new_col_names],label_test), 
                epochs=n_epochs,
                batch_size=n_batch,
                shuffle=to_shuffle)

Train on 2746 samples, validate on 1177 samples
Epoch 1/5
2746/2746 [==============================] - 4s 1ms/step - loss: 0.5333 - acc: 0.7775 - keras_precision: 0.7775 - keras_recall: 0.7775 - keras_auc: 0.7662 - val_loss: 0.5303 - val_acc: 0.7774 - val_keras_precision: 0.7774 - val_keras_recall: 0.7774 - val_keras_auc: 0.7739
Epoch 2/5
2746/2746 [==============================] - 4s 1ms/step - loss: 0.5307 - acc: 0.7775 - keras_precision: 0.7775 - keras_recall: 0.7775 - keras_auc: 0.7744 - val_loss: 0.5301 - val_acc: 0.7774 - val_keras_precision: 0.7774 - val_keras_recall: 0.7774 - val_keras_auc: 0.7738
Epoch 3/5
2746/2746 [==============================] - 3s 1ms/step - loss: 0.5304 - acc: 0.7775 - keras_precision: 0.7775 - keras_recall: 0.7775 - keras_auc: 0.7733 - val_loss: 0.5301 - val_acc: 0.7774 - val_keras_precision: 0.7774 - val_keras_recall: 0.7774 - val_keras_auc: 0.7741
Epoch 4/5
2746/2746 [==============================] - 4s 1ms/step - loss: 0.5302 - acc: 0.7775 - keras

In [45]:
cnn_1.name = "cnn_1"
cnn_2.name = "cnn_2"
cnn_3.name = "cnn_3"
auto_encoder_1.name = "auto_encoder_1"
ann_models = [cnn_1, cnn_2, cnn_3, auto_encoder_1]

In [65]:
for model in ann_models:
    print("##########################", model.name, "##########################")
    print(model.summary())

########################## cnn_1 ##########################
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          100000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 100)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 96, 64)            32064     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 24, 64)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 198,266
Trainable params: 198,266
Non-trainable params: 0
__________

# TODO: fix results table 

In [110]:
new_rows = []

for model in ann_models:        
    for metric_key, values in model.history.history.items():
        for value in values: # each epoch
            
            row = {"model": model.name, 
                   "epoch":values.index(value) + 1,
                  "metric":metric_key, 
                  "value":value, }
        new_rows.append(row)
        
        
ann_results_df = pd.DataFrame(new_rows)
ann_results_df.head(len(ann_results_df))

,epoch,metric,model,value
0,5,val_loss,cnn_1,0.486026
1,5,val_acc,cnn_1,0.853866
2,5,val_keras_precision,cnn_1,0.853866
3,5,val_keras_recall,cnn_1,0.853866
4,5,val_keras_auc,cnn_1,0.960798
5,5,loss,cnn_1,0.072545
6,5,acc,cnn_1,0.975237
7,5,keras_precision,cnn_1,0.975237
8,5,keras_recall,cnn_1,0.975237
9,5,keras_auc,cnn_1,0.958070


In [97]:
last_epoch = ann_results_df.loc[ann_results_df['epoch'] == '5']
last_epoch.head(len(last_epoch))

,epoch,metric,model,value
4,5,val_loss,cnn_1,0.486026
9,5,val_acc,cnn_1,0.853866
14,5,val_keras_precision,cnn_1,0.853866
19,5,val_keras_recall,cnn_1,0.853866
24,5,val_keras_auc,cnn_1,0.960798
29,5,loss,cnn_1,0.072545
34,5,acc,cnn_1,0.975237
39,5,keras_precision,cnn_1,0.975237
44,5,keras_recall,cnn_1,0.975237
49,5,keras_auc,cnn_1,0.958070


## Making Prections

In [90]:
full_data ='../data/ALL_CANDIDATES.json'
pred_df = load_data.getJSONData(full_data)
pred_df.head()

,annotation,fileID,text
0,NON_permission_statement.,1,a copy of this entire consent form will be giv...
1,NON_permission_statement,1,"""what if i change my mind about participating ..."
2,NON_permission_statement,1,ask your study doctor for more information abo...
3,NON_permission_statement,1,newborn screening required additional language...
4,NON_permission_statement,1,"""if you are a healthy volunteer, no direct med..."


In [91]:
vocabulary_size = 1000
maxlen = 100

tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(pred_df['text'])
sequences = tokenizer.texts_to_sequences(pred_df['text'])
data = pad_sequences(sequences, maxlen=maxlen)

pred_df['data'] = data.tolist()

In [92]:
new_col_names = ['seq_posi' + str(i) for i in range(0,maxlen)]
pos_seq_df = pd.DataFrame(pred_df['data'].values.tolist(), columns=new_col_names)
pred_df = pd.concat([pred_df, pos_seq_df], axis=1)

In [93]:
predictions = cnn_1.predict_classes(pred_df[new_col_names])

In [94]:
print('Total Positive Predictions:', sum(predictions))
print('Number of Cases:', len(predictions))
print('Percentage Predicted Positive: ', sum(predictions)/len(predictions))


Total Positive Predictions: 3882
Number of Cases: 20000
Percentage Predicted Positive:  0.1941


In [111]:
positives_df = df.loc[df['predictions'] == 1]

KeyError: 'predictions'

In [112]:
for idx, row in positives_df.head(10).iterrows():
    print(row['fileID'], row['text'], '\n')
    

NameError: name 'positives_df' is not defined